# Naive Bayes를 활용하여 유사 문서 검색

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',
                                  categories=categories, shuffle=True,
                                  random_state=13)

In [2]:
twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [3]:
# 데이터셋 설명
print(twenty_train['DESCR'])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [4]:
# 타겟 이름과 데이터 갯수
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [5]:
len(twenty_train.data)

2257

In [6]:
print(twenty_train.data[0])

From: geb@cs.pitt.edu (Gordon Banks)
Subject: Re: Update (Help!) [was "What is This [Is it Lyme's?]"]
Article-I.D.: pitt.19436
Reply-To: geb@cs.pitt.edu (Gordon Banks)
Organization: Univ. of Pittsburgh Computer Science
Lines: 42

In article <1993Mar29.181958.3224@equator.com> jod@equator.com (John Setel O'Donnell) writes:
>
>I shouldn't have to be posting here.  Physicians should know the Lyme
>literature beyond Steere & co's denial merry-go-round.  Patients
>should get correctly diagnosed and treated.
>

Why do you think Steere is doing this?  Isn't he acting in good faith?
After all, as the "discoverer" of Lyme for all intents and purposes,
the more famous Lyme gets, the more famous Steere gets.  I don't
see the ulterior motive here.  It is easy for me to see it the
those physicians who call everything lyme and treat everything.
There is a lot of money involved.

>I'm a computer engineer, not a doctor (,Jim).  I was building a 
>computer manufacturing company when I got Lyme. I lost 

In [7]:
# 타겟 확인
print(twenty_train.target_names[twenty_train.target[0]])

sci.med


In [8]:
twenty_train.target[:10]

array([2, 1, 1, 3, 2, 3, 0, 3, 2, 0])

## CountVectorize
* 희소행렬로 나타내는 함수 - toarray()

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [10]:
X_train_counts.toarray()[0]
len(X_train_counts.toarray()[0])

35788

In [12]:
import numpy as np

np.sum(X_train_counts.toarray()[0])

320

## TF-IDF 적용

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

## Multinomial Naive Bayes 적용

In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, twenty_train.target)

In [16]:
# 테스트
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## pipeline

In [17]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

In [19]:
# 학습 후 test accuracy 적용
text_clf.fit(twenty_train.data, twenty_train.target)

twenty_test = fetch_20newsgroups(subset='test',
                                  categories=categories, shuffle=True,
                                  random_state=13)
docs_test = twenty_train.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9693741677762983

## Classification Report

In [20]:
from sklearn.metrics import classification_report

print(classification_report(twenty_test.target, predicted,
      target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       1.00      0.90      0.95       319
         comp.graphics       1.00      0.97      0.98       389
               sci.med       1.00      0.99      1.00       396
soc.religion.christian       0.90      1.00      0.95       398

              accuracy                           0.97      1502
             macro avg       0.97      0.97      0.97      1502
          weighted avg       0.97      0.97      0.97      1502



## Confusion matrix

In [21]:
from sklearn.metrics import confusion_matrix

confusion_matrix(twenty_test.target, predicted)

array([[286,   0,   1,  32],
       [  0, 378,   0,  11],
       [  0,   1, 394,   1],
       [  0,   0,   0, 398]])

# SMS Spam Collection Dataset